In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
from keras.utils import normalize
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import os
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
import numpy as np
import random
from tqdm import tqdm
import json
# import segmentation_models as sm
from tensorflow.keras.metrics import MeanIoU, Recall, Precision, BinaryAccuracy
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
learning_rate = 0.001
from keras import backend as K
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Add, BatchNormalization, Activation
import numpy as np
# from tensorflow.keras.activations import leaky_relu
# from focal_loss import sparse_categorical_focal_loss
# from focal_loss import SparseCategoricalFocalLoss
# from tensorflow.keras.metrics import DiceCoefficient

In [3]:
# Correct recall for tf
def recall_tf(y_true, y_pred):
    Y_true = tf.dtypes.cast(y_true, tf.float32)
    Y_true = K.flatten(Y_true)
    Y_pred = tf.dtypes.cast(tf.math.greater(y_pred, 0.5), tf.float32)
    Y_pred = K.flatten(Y_pred)

    true_positives = tf.reduce_sum(Y_true * Y_pred)
    possible_positives = tf.reduce_sum(Y_true)
    recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
    return recall

In [4]:
# Correct tf precision
def precision_tf(y_true, y_pred):
    Y_true = tf.dtypes.cast(y_true, tf.float32)
    Y_true = K.flatten(Y_true)
    Y_pred = tf.dtypes.cast(tf.math.greater(y_pred, 0.5), tf.float32)
    Y_pred = K.flatten(Y_pred)

    true_positives = tf.reduce_sum(tf.round(tf.clip_by_value(Y_true * Y_pred, 0, 1)))
    print(true_positives)
    predicted_positives = tf.reduce_sum(tf.round(tf.clip_by_value(Y_pred, 0, 1)))
    print(predicted_positives)
    precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
    return precision

In [5]:
# Correct tf dicecoefficient
def dice_coefficient_tf(y_true, y_pred):
    Y_true = tf.dtypes.cast(y_true, tf.float32)
    Y_true = K.flatten(Y_true)
    Y_pred = tf.dtypes.cast(tf.math.greater(y_pred, 0.5), tf.float32)
    Y_pred = K.flatten(Y_pred)

    smooth = 1e-5

    intersection = tf.reduce_sum(Y_true * Y_pred)
    union = tf.reduce_sum(Y_true) + tf.reduce_sum(Y_pred)
    return (2.0 * intersection + smooth) / (union + smooth)


In [6]:
def dice_coefficient_for_loss(y_true, y_pred):
    Y_true = tf.dtypes.cast(y_true, tf.float32)
    Y_true = K.flatten(Y_true)

    Y_pred = K.flatten(y_pred)

    smooth = 1e-5

    intersection = K.sum(Y_true * Y_pred)

    union = K.sum(Y_true) + K.sum(Y_pred)
    return (2.0 * intersection + smooth) / (union + smooth)


In [7]:
def dice_coef_loss(y_true, y_pred):
    loss = - dice_coefficient_for_loss(y_true, y_pred)
    return loss

In [8]:
# Pre-Processing the training data
n_classes = 3
X_train = np.load('/content/drive/MyDrive/Dataset/liver_dataset_improoved/train2/X_train128.npy')
Y_train = np.load('/content/drive/MyDrive/Dataset/liver_dataset_improoved/train2/Y_train128.npy')

X_test = np.load('/content/drive/MyDrive/Dataset/liver_dataset_improoved/X_test128.npy')
Y_test = np.load('/content/drive/MyDrive/Dataset/liver_dataset_improoved/Y_test128.npy')

X_train = np.expand_dims(X_train, axis = 3)
#Normalizing X_train values only
X_train = normalize(X_train, axis = 1)

X_test = np.expand_dims(X_test, axis = 3)
X_test = normalize(X_test, axis = 1)


Y_train = np.expand_dims(Y_train, axis = 3)
Y_test = np.expand_dims(Y_test, axis = 3)

print("Training Data")
print(X_train.shape)
print(np.unique(X_train))

print(Y_train.shape)
print(np.unique(Y_train))

# Connverting the Y_train into categorical
Y_train_cat = to_categorical(Y_train, num_classes=n_classes)
Y_train_cat = Y_train_cat.reshape((Y_train.shape[0], Y_train.shape[1], Y_train.shape[2], n_classes))
print(Y_train_cat.shape)

Y_test_cat = to_categorical(Y_test, num_classes=n_classes)
Y_test_cat = Y_test_cat.reshape((Y_test.shape[0], Y_test.shape[1], Y_test.shape[2], n_classes))
print(Y_test_cat.shape)

Training Data
(10000, 128, 128, 1)
[0.00000000e+00 5.22079475e-04 5.32547051e-04 ... 9.99986437e-01
 9.99988443e-01 1.00000000e+00]
(10000, 128, 128, 1)
[0 1 2]
(10000, 128, 128, 3)
(1916, 128, 128, 3)


In [9]:
IMG_HEIGHT = 128
IMG_WIDTH = 128
IMG_CHANNELS = 1
n_classes = 3  # 0, 1, 2
#Model Building
seed = 42
np.random.seed = seed
# 1) Creating the input layer. The input layer are the images of 128*128 size and channel = 1
inputs = tf.keras.layers.Input((IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS))

s = inputs

# BLOCK 1
c1 = Conv2D(16, (3,3), strides= (1, 1), padding='same')(s)
c1 =  Activation('relu')(c1)
c1 = Conv2D(16, (3,3), strides= (1, 1), padding='same')(c1)
c1 = BatchNormalization()(c1)
c1 =  Activation('relu')(c1)    #-->


#Jump Connection
conn1 = Activation('leaky_relu')(c1) #--->
conn1 = BatchNormalization()(conn1)
conn1 = tf.keras.layers.Dropout (0.1)(conn1)

pool1 = Conv2D(filters = 16 ,  # Set the desired number of filters
               kernel_size=(5, 5),
               padding='same',  # Padding of 2 is achieved with 'valid' padding
               strides=(1, 1))(conn1)
x1 = Activation('relu')(pool1)
x1 = Conv2D(16, (3,3), strides= (1, 1), padding='same')(x1) #-->

sum1 = Add()([conn1, x1])

pool2 = Conv2D(filters = 16 ,  # Set the desired number of filters
               kernel_size=(5, 5),
               padding='same',  # Padding of 2 is achieved with 'valid' padding
               strides=(1, 1))(x1)
x2 = Activation('relu')(pool2)
x2 = Conv2D(16, (3,3), strides= (1, 1), padding='same')(x2) #-->

sum2 = Add()([sum1, x2]) #->
#---------------------------------------------------------------------------------------------#

#2nd Block
c2 = Conv2D(32, (3,3), strides= (1, 1), padding='same')(c1)
c2 =  Activation('relu')(c2)
c2 = Conv2D(32, (3,3), strides= (1, 1), padding='same')(c2)
c2 = BatchNormalization()(c2)
c2 =  Activation('relu')(c2) #-->

#Jump Connection
conn2 = Activation('leaky_relu')(c2) #--->
conn2 = BatchNormalization()(conn2)
conn2 = tf.keras.layers.Dropout (0.1)(conn2)

pool3 = Conv2D(filters = 32 ,  # Set the desired number of filters
               kernel_size=(5, 5),
               padding='same',  # Padding of 2 is achieved with 'valid' padding
               strides=(1, 1))(conn2)
x3 = Activation('relu')(pool3)
x3 = Conv2D(32, (3,3), strides= (1, 1), padding='same')(x3) #-->

sum3 = Add()([conn2, x3])

pool4 = Conv2D(filters = 32 ,  # Set the desired number of filters
               kernel_size=(5, 5),
               padding='same',  # Padding of 2 is achieved with 'valid' padding
               strides=(1, 1))(x3)
x4 = Activation('relu')(pool4)
x4 = Conv2D(32, (3,3), strides= (1, 1), padding='same')(x4) #-->

sum4 = Add()([sum3, x4]) #->
#----------------------------------------------------------------------------------------------------#

#3rd Block
c3 = Conv2D(64, (3,3), strides= (1, 1), padding='same')(c2)
c3 =  Activation('relu')(c3)
c3 = Conv2D(64, (3,3), strides= (1, 1), padding='same')(c3)
c3 = BatchNormalization()(c3)
c3 =  Activation('relu')(c3) #-->


conn3 = Activation('leaky_relu')(c3) #--->
conn3 = BatchNormalization()(conn3)
conn3 = tf.keras.layers.Dropout (0.1)(conn3)

pool5 = Conv2D(filters = 64 ,  # Set the desired number of filters
               kernel_size=(5, 5),
               padding='same',  # Padding of 2 is achieved with 'valid' padding
               strides=(1, 1))(conn3)
x5 = Activation('relu')(pool5)
x5 = Conv2D(64, (3,3), strides= (1, 1), padding='same')(x5) #-->

sum5 = Add()([conn3, x5])

pool6 = Conv2D(filters = 64 ,  # Set the desired number of filters
               kernel_size=(5, 5),
               padding='same',  # Padding of 2 is achieved with 'valid' padding
               strides=(1, 1))(x5)
x6 = Activation('relu')(pool6)
x6 = Conv2D(64, (3,3), strides= (1, 1), padding='same')(x6) #-->

sum6 = Add()([sum5, x6]) #->

#----------------------------------------------------------------------------------------------------#

#4th Block
c4 = Conv2D(128, (3,3), strides= (1, 1), padding='same')(c3)
c4 =  Activation('relu')(c4)
c4 = Conv2D(128, (3,3), strides= (1, 1), padding='same')(c4)
c4 = BatchNormalization()(c4)
c4 =  Activation('relu')(c4) #-->


conn4 = Activation('leaky_relu')(c4) #--->
conn4 = BatchNormalization()(conn4)
conn4 = tf.keras.layers.Dropout (0.1)(conn4)

pool7 = Conv2D(filters = 128 ,  # Set the desired number of filters
               kernel_size=(5, 5),
               padding='same',  # Padding of 2 is achieved with 'valid' padding
               strides=(1, 1))(conn4)
x7 = Activation('relu')(pool7)
x7 = Conv2D(128, (3,3), strides= (1, 1), padding='same')(x7) #-->

sum7 = Add()([conn4, x7])

pool8 = Conv2D(filters = 128 ,  # Set the desired number of filters
               kernel_size=(5, 5),
               padding='same',  # Padding of 2 is achieved with 'valid' padding
               strides=(1, 1))(x7)
x8 = Activation('relu')(pool8)
x8 = Conv2D(128, (3,3), strides= (1, 1), padding='same')(x8) #-->

sum8 = Add()([sum7, x8]) #->

#----------------------------------------------------------------------------------------------------#

#5th Block
c5 = Conv2D(256, (3,3), strides= (1, 1), padding='same')(c4)
c5 =  Activation('relu')(c5)
c5 = Conv2D(256, (3,3), strides= (1, 1), padding='same')(c5)
c5 = BatchNormalization()(c5)
c5 =  Activation('relu')(c5) #-->


p1 = tf.keras.layers.MaxPooling2D(2,2)(c5)
u6 = tf.keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(p1)
u6 = tf.keras.layers.concatenate([u6, sum8])
c6 = tf.keras.layers.Conv2D(128, (3,3), activation= 'relu', kernel_initializer= 'he_normal', padding= 'same')(u6)
c6 = tf. keras.layers.Dropout (0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)


p2 = tf.keras.layers.MaxPooling2D(2,2)(c6)
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(p2)
u7 = tf.keras. layers. concatenate ([u7, sum6])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf. keras.layers.Dropout (0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

p3 = tf.keras.layers.MaxPooling2D(2,2)(c7)
u8 = tf. keras.layers .Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(p3)
u8 = tf. keras.layers. concatenate ([u8, sum4])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout (0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

p4 = tf.keras.layers.MaxPooling2D(2,2)(c8)
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(p4)
u9 = tf.keras.layers. concatenate ([u9, sum2], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf. keras.layers. Dropout (0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)



outputs = tf.keras.layers.Conv2D(n_classes, (1, 1), activation= 'softmax')(c9)

model = tf.keras.Model(inputs= [inputs], outputs= [outputs])
model.summary()

model.compile(
    optimizer=Adam(learning_rate),
    loss= dice_coef_loss,
    metrics=[
        BinaryAccuracy(name='accuracy'),  # Binary Accuracy
        MeanIoU(num_classes=3, name='iou'),  # Intersection Over Union (IoU)
        recall_tf,  # Sensitivity
        precision_tf,
         tf.keras.metrics.BinaryIoU(
            threshold=0.5,
            name='binaryIoU',
            dtype=None
        ),
        Recall(name='recall_inbuilt'),
        Precision(name='precision_inbuilt'),
        dice_coefficient_tf  # Custom dice coefficient function
    ]
)

checkpointer = tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/Project7thSem/Liver/NewArchLeaky/DiceLoss/liver_checkpoint.h5', verbose=1, save_best_only=True)


callbacks = [
              tf.keras.callbacks.EarlyStopping(patience = 5, monitor = 'val_loss'),
              tf.keras.callbacks.TensorBoard(log_dir = '/content/drive/MyDrive/Project7thSem/Liver/NewArchLeaky/DiceLoss'),
              checkpointer
]

# Saving the accuracy as the model gets trained

results = model.fit(X_train, Y_train_cat, validation_split=0.1, batch_size=16, epochs=100,  callbacks=callbacks)
model.save('/content/drive/MyDrive/Project7thSem/Liver/NewArchLeaky/DiceLoss/unet_liver_model128.h5')


# Saving the data
with open('/content/drive/MyDrive/Project7thSem/Liver/NewArchLeaky/DiceLoss/training_history128.json', 'w') as file:
     json.dump(results.history, file)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 16)         160       ['input_1[0][0]']             
                                                                                                  
 activation (Activation)     (None, 128, 128, 16)         0         ['conv2d[0][0]']              
                                                                                                  
 conv2d_1 (Conv2D)           (None, 128, 128, 16)         2320      ['activation[0][0]']          
                                                                                              

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


563/563 [==============================] - 624s 1s/step - loss: -0.9280 - accuracy: 0.9522 - iou: 0.8973 - recall_tf: 0.9280 - precision_tf: 0.9282 - binaryIoU: 0.8984 - recall_inbuilt: 0.9279 - precision_inbuilt: 0.9285 - dice_coefficient_tf: 0.9281 - val_loss: -0.9300 - val_accuracy: 0.9533 - val_iou: 0.9008 - val_recall_tf: 0.9299 - val_precision_tf: 0.9299 - val_binaryIoU: 0.9008 - val_recall_inbuilt: 0.9300 - val_precision_inbuilt: 0.9300 - val_dice_coefficient_tf: 0.9299
Epoch 2/100
563/563 [==============================] - ETA: 0s - loss: -0.9292 - accuracy: 0.9528 - iou: 0.8997 - recall_tf: 0.9292 - precision_tf: 0.9292 - binaryIoU: 0.8997 - recall_inbuilt: 0.9292 - precision_inbuilt: 0.9292 - dice_coefficient_tf: 0.9292
Epoch 2: val_loss did not improve from -0.93000
563/563 [==============================] - 561s 997ms/step - loss: -0.9292 - accuracy: 0.9528 - iou: 0.8997 - recall_tf: 0.9292 - precision_tf: 0.9292 - binaryIoU: 0.8997 - recall_inbuilt: 0.9292 - precision_inbu

KeyboardInterrupt: ignored